In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import hiplot as hip
import neptune
import numpy as np
import pandas as pd
import seaborn as sns

In [ ]:
sns.set(style="darkgrid")

In [ ]:
p = neptune.init('neptune-workshops/AII-Optimali')

# Fetch DataFrame with experiments data

In [ ]:
experiments_df = p.get_leaderboard(tag='classification')
cols = ['id', 'owner',
       'channel_eval_accuracy', 'channel_eval_loss', 'channel_learning_rate',
       'parameter_batch_size', 'parameter_dense_units', 'parameter_dropout',
       'parameter_learning_rate', 'parameter_optimizer']

In [ ]:
experiments_df = experiments_df[cols]

# Check hardware utilization

In [ ]:
exps_list = p.get_experiments(experiments_df.id.to_list())
epoch_data_df = pd.DataFrame(columns=['id', 'x_cpu', 'y_cpu', 'x_ram', 'y_ram'])
for exp in exps_list:
    exp_data_df = exp.get_hardware_utilization()
    exp_data_df = exp_data_df.apply(lambda x: x//1000 if x.name == 'x_cpu' or x.name == 'x_ram' else x)
    exp_data_df.insert(loc=0, column='id', value=exp.id)
    epoch_data_df = epoch_data_df.append(exp_data_df)
epoch_data_df.head()

# Plot CPU utilization

In [ ]:
sns.relplot(x='x_cpu', y='y_cpu', kind='line', height=10, data=epoch_data_df);

# Plot RAM utilization

In [ ]:
sns.relplot(x='x_ram', y='y_ram', kind='line', height=10, data=epoch_data_df);

# HiPlot

In [ ]:
input_to_hiplot = experiments_df.T.to_dict().values()

In [ ]:
hip.Experiment.from_iterable(input_to_hiplot).display()